為什麼這個老是報錯? 即便從別的可以正常執行的複製過來還是報錯?

結果是因為 

gemini = ChatGoogleGenerativeAI(model = "gemini-pro", verbose = True, temperature = 0.6, google_api_key = os.environ["GOOGLE_API_KEY"]),

# 最後面多了一個","造成的，切記！！！！！！！

In [1]:
import os
from crewai import Agent, Task, Crew, Process

In [2]:
import textwrap
from IPython.display import HTML, display, Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
# 設定4個 LLM
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI
# crewai 使用 Ollama 的方法：windows 只需要打開 ubuntu 視窗即可，不需要執行 litellm；Mac 則甚至不需要啟動任何程式，只要有安裝好 Ollama 即可調用。
# 與 autogen 不同，autogen 要調用 Ollama 需要執行 litellm，多一道步驟，比較麻煩
gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
gpt4 = ChatOpenAI(model_name="gpt-4", temperature=0.7),
# Ollama 大模型，可以看要用哪一個
ollama = Ollama(model="openhermes_assistant")  # 這是用 modelfile 建立的
ollama_openhermes = Ollama(model="openhermes")

gemini = ChatGoogleGenerativeAI(model="gemini-pro",
                             verbose = True,
                             temperature = 0.6,
                             google_api_key = os.environ["GOOGLE_API_KEY"]
                             )


In [4]:
# 設定工具 tools
# Make sure to Install duckduckgo-search for this example:
# !pip install -U duckduckgo-search
# 最簡易型的工具定義方式
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

# 從 tools 模組中引入需要的工具
from tools.browser_tools import BrowserTools
from tools.calculator_tools import CalculatorTools
from tools.file_tools import FileTools
from tools.search_tools import SearchTools
from tools.sec_tools import SECTools
# 下面添加 tools 時，必須用 類別.方法 加入工具
# BrowserTools.scrape_and_summarize_website
# CalculatorTools.calculate
# FileTools.write_file
# SearchTools.search_internet、SearchTools.search_news
# SECTools.search_10q、SECTools.search_10k
# 下面的是依樣畫葫蘆自行加入2行定義類別，親測可以匯入
from tools.find_papers_arxiv import FindPapersArxiv
from tools.get_top_hackernews_stories import GetTopHackernewsStories
# 然後就可以用下面的函式來使用工具，已測試可以~~~
# FindPapersArxiv.search_arxiv
# GetTopHackernewsStories.get_top_hackernews_stories

In [5]:
# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting 
  actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_tool],
  debug_mode=True,
  # You can pass an optional llm attribute specifying what mode you wanna use.
  # It can be a local model through Ollama / LM Studio or a remote
  # model like OpenAI, Mistral, Antrophic or others (https://python.langchain.com/docs/integrations/llms/)
  #
  # Examples:
  #
  # from langchain_community.llms import Ollama
  # llm=gemini # was defined above in the file
  #
  # from langchain_openai import ChatOpenAI
  # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7)
)
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  debug_mode=True,
  # llm=gemini
  # (optional) llm=ollama_llm
)

In [6]:
researcher.llm = ollama
writer.llm = gemini

In [8]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report""",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.
  Your answer MUST be the full blog post of at least 5 paragraphs and finally using taditional chinese writing.""",
  agent=writer
)

In [9]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

In [10]:
# Get your crew to work!
result = crew.kickoff()

# print("######################")
# print(result)
to_markdown(result)

[DEBUG]: Working Agent: Senior Research Analyst
[INFO]: Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report


> Entering new CrewAgentExecutor chain...
Final Answer: AI Advancements in 2024 Analysis Report

Introduction:
AI has been rapidly evolving and making its presence felt across various industries. As we approach 2024, the advancements in AI are expected to be even more significant, disrupting industries and reshaping our lives. This report aims to provide a comprehensive analysis of the latest trends, breakthrough technologies, and potential industry impacts in the field of AI by 2024.

1. Advancements in Machine Learning:

Machine learning algorithms will continue to improve in their ability to process complex data sets and make accurate predictions. The development of more advanced deep learning models and th

> **2024年人工智能的重大突破，引領時代的先鋒科技**
> 
> 人工智能（AI）正在迅速發展，並在各個行業中發揮著越來越重要的作用。隨著2024年的臨近，人工智能的進步預計將更加顯著，顛覆行業並重塑我們的生活。這篇文章旨在全面分析2024年人工智能領域的最新趨勢、突破性技術和潛在的行業影響。
> 
> **1.機器學習的進步**
> 
> 機器學習演算法將繼續提高處理複雜數據集和做出準確預測的能力。更先進的深度學習模型的開發和量子計算與機器學習系統的整合，預計將在自然語言處理、電腦視覺和語音識別等領域取得重大突破。
> 
> **2.邊緣人工智能（Edge AI）**
> 
> 邊緣人工智能的增長將使處理能力有限的設備能夠執行以前只能在強大伺服器上才能完成的任務。這將導致人工智能支援的物聯網（IoT）設備的激增，使它們更自主並能對本地環境做出反應。
> 
> **3.可解釋的人工智能（Explainable AI）**
> 
> 隨著人工智能繼續融入我們生活的各個方面，透明度和可解釋性的需求變得至關重要。在2024年，我們將看到可解釋人工智能（XAI）的進步，使人類更容易理解人工智能系統是如何做出決定的。這將有助於建立對人工智能驅動技術的信任，並防止潛在的誤用或意外後果。
> 
> **4.機器人和自主系統**
> 
> 人工智能與機器人和自主系統的整合將在製造、物流和運輸等領域取得重大進展。我們可以預期在各個行業中廣泛採用自動駕駛汽車、無人機和機器人，從而提高效率和生產力。
> 
> **5.人機協作**
> 
> 隨著人工智能的繼續發展，人機協作將成為一個日益關注的焦點。這涉及設計與人類並肩工作的AI系統，增強人類的能力，而不是取代人類。我們可以預期看到人工智能工具的開發，這些工具可以協助決策、創造力和問題解決，從而提高生產力和創造就業機會。
> 
> **6.人工智能治理**
> 
> 隨著人工智能更多地融入我們的生活，對負責任的人工智能治理的需求將不斷增長。這包括解決數據隱私、安全和決策公平性等問題。我們可以預期看到監管框架和行業標準的制定，旨在確保人工智能技術的負責任使用。
> 
> **7.醫療保健和醫療進步**
> 
> 人工智能在醫療保健中的整合將在診斷、藥物發現和個性化醫療等領域取得重大進展。人工智能驅動的工具將幫助醫生做出更準確的診斷、開發新的治療方法，並根據個別患者數據提供更有針對性的治療。
> 
> **結論**
> 
> 人工智能將徹底改變各行各業，並改變我們的生活和工作方式。預計2024年的進步將非常顯著，機器學習、邊緣人工智能、可解釋人工智能、機器人技術、人機協作、人工智能治理和醫療保健等領域都將取得突破。隨著我們走向更加人工智能驅動的未來，至關重要的是，我們要解決挑戰並確保這些技術的負責任開發和實施。